In [39]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df = pd.read_csv('/Users/t2023-m0069/Desktop/Github/Omakase/Courses.csv')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641138 entries, 0 to 641137
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              641138 non-null  int64  
 1   course_id          641138 non-null  object 
 2   userid_DI          641138 non-null  object 
 3   registered         641138 non-null  int64  
 4   viewed             641138 non-null  int64  
 5   explored           641138 non-null  int64  
 6   certified          641138 non-null  int64  
 7   final_cc_cname_DI  641138 non-null  object 
 8   LoE_DI             535130 non-null  object 
 9   YoB                544533 non-null  float64
 10  gender             554332 non-null  object 
 11  grade              592766 non-null  object 
 12  start_time_DI      641138 non-null  object 
 13  last_event_DI      462184 non-null  object 
 14  nevents            441987 non-null  float64
 15  ndays_act          478395 non-null  float64
 16  np

In [41]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df_raw = pd.read_csv("Courses.csv")  # 경로는 필요에 따라 수정
original_len = len(df_raw)

df = df_raw.copy()
removal_log = {}

# 날짜 변환
df['start_time_DI'] = pd.to_datetime(df['start_time_DI'], errors='coerce')
df['last_event_DI'] = pd.to_datetime(df['last_event_DI'], errors='coerce')

# grade 전처리
df['grade'] = pd.to_numeric(df['grade'].replace(' ', np.nan), errors='coerce')
df['grade'] = df['grade'].replace(1.01, 1.0)

# viewed=False인데 explored or certified=True → 삭제
mask_unseen_completed = (df['viewed'] == False) & ((df['explored'] == True) | (df['certified'] == True))
removal_log['viewed=False but explored/certified=True'] = mask_unseen_completed.sum()
df = df[~mask_unseen_completed]

# viewed=False이고 활동지표 모두 채워진 경우 삭제
conditions = ['nplay_video', 'nchapters', 'nevents', 'ndays_act']
cond_all_filled = (df['viewed'] == False) & df[conditions].notnull().all(axis=1)
removal_log['viewed=False and all activity filled'] = cond_all_filled.sum()
df = df[~cond_all_filled]

# viewed=False이고 일부만 채워진 경우 → 0으로 대체
cond_partial_filled = (df['viewed'] == False) & df[conditions].notnull().any(axis=1)
df.loc[cond_partial_filled, conditions] = df.loc[cond_partial_filled, conditions].fillna(0)

# 날짜 오류 제거
date_error = (df['last_event_DI'] < df['start_time_DI']) & df['last_event_DI'].notna() & df['start_time_DI'].notna()
removal_log['last_event < start_time'] = date_error.sum()
df = df[~date_error]

# start_time과 last_event 모두 결측 → 제거
both_missing = df['start_time_DI'].isna() & df['last_event_DI'].isna()
removal_log['both start_time and last_event missing'] = both_missing.sum()
df = df[~both_missing]

# YoB 이상치 제거
q3 = df['YoB'].quantile(0.75)
iqr = q3 - df['YoB'].quantile(0.25)
upper_bound = q3 + 1.5 * iqr
yob_outliers = df['YoB'] > upper_bound
removal_log[f'YoB > {upper_bound:.1f}'] = yob_outliers.sum()
df = df[~yob_outliers]

# nevents 이상치 제거
nevents_outlier = df['nevents'] == 197757
removal_log['nevents == 197757'] = nevents_outlier.sum()
df = df[~nevents_outlier]

# 최종 정리
removal_log['TOTAL REMOVED'] = original_len - len(df)
removal_log['FINAL REMAINING'] = len(df)

# 결과 출력
print("\n📊 조건별 삭제된 행 수")
for k, v in removal_log.items():
    print(f"- {k}: {v}")



📊 조건별 삭제된 행 수
- viewed=False but explored/certified=True: 8
- viewed=False and all activity filled: 1
- last_event < start_time: 1443
- both start_time and last_event missing: 0
- YoB > 2004.5: 608
- nevents == 197757: 1
- TOTAL REMOVED: 2061
- FINAL REMAINING: 639077


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 639077 entries, 0 to 641137
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   index              639077 non-null  int64         
 1   course_id          639077 non-null  object        
 2   userid_DI          639077 non-null  object        
 3   registered         639077 non-null  int64         
 4   viewed             639077 non-null  int64         
 5   explored           639077 non-null  int64         
 6   certified          639077 non-null  int64         
 7   final_cc_cname_DI  639077 non-null  object        
 8   LoE_DI             533404 non-null  object        
 9   YoB                542781 non-null  float64       
 10  gender             552548 non-null  object        
 11  grade              582339 non-null  float64       
 12  start_time_DI      639077 non-null  datetime64[ns]
 13  last_event_DI      460454 non-null  datetime64[ns